In [6]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel , RunnableBranch , RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
load_dotenv()

# -------------------------
# 1️⃣ Pydantic Schema
# -------------------------
class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(
        description="Give the sentiment of the Feedback"
    )
llm = ChatOllama(
    model="qwen3:latest",
    temperature=0
)

In [7]:
user_input = "This is a terrible phone" 

In [8]:
pydantic_feedback_parser = PydanticOutputParser(pydantic_object=Feedback)
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into positive or negative:\n"
             "{feedback}\n\n{format_instructions}",
    input_variables=['feedback'],
    partial_variables={'format_instructions': pydantic_feedback_parser.get_format_instructions()}
)
print(prompt1.format(feedback=user_input))

Classify the sentiment of the following feedback text into positive or negative:
This is a terrible phone

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sentiment": {"description": "Give the sentiment of the Feedback", "enum": ["positive", "negative"], "title": "Sentiment", "type": "string"}}, "required": ["sentiment"]}
```


In [10]:
chain = prompt1 | llm | pydantic_feedback_parser

classifier_output = chain.invoke({"feedback": user_input})

In [53]:
classifier_output

Feedback(sentiment='negative')

In [11]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback'] ) 

In [12]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | llm | StrOutputParser()),
    (lambda x:x.sentiment == 'negative', prompt3 | llm | StrOutputParser()),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [56]:
branch_chain.invoke(classifier_output)

'{\n  "response": "We\'re sorry to hear that you\'re experiencing issues with your order. We value your feedback and apologize for the inconvenience caused. Could you please share more details about the problem you encountered? Our team is here to help resolve this promptly, whether it\'s a refund, replacement, or other solution. Your satisfaction is important to us, and we appreciate the opportunity to improve. Please let us know how we can assist further!"\n}'

In [13]:
chain = chain | branch_chain 
result = chain.invoke({'feedback': 'This is a beautiful phone'})
print(result)


**Response:**  
"Thank you so much for your kind words! I'm really glad to hear that. How are you doing? If there's anything else I can assist you with, feel free to let me know! 😊"  

This response acknowledges the feedback warmly, expresses gratitude, and opens the door for further engagement. It maintains a friendly and approachable tone.


In [ ]:
for chunk in chain.stream({'feedback': 'This is a beautiful phone'}):
    #print("---------")
    print(chunk, end="", flush=True)

---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------
---------


* qwen3:latest model is generating Pydantic strOutput 
* lambda:3.2 latest model is sometimes generates the pydantic output and sometimes not
    1) because is not a thinking model
    2) It does not even support Tools

# manually running

In [58]:
feedback_text = "This is a terrible phone"
str_output_parser = StrOutputParser()
# Step 2: Branch manually
if classifier_output.sentiment == "positive":
    formatted_response_prompt = prompt2.format(feedback=feedback_text)
elif classifier_output.sentiment == "negative":
    formatted_response_prompt = prompt3.format(feedback=feedback_text)
else:
    print("Could not find sentiment")
    exit()

# Step 3: Generate response
response_raw = llm.invoke(formatted_response_prompt)
final_response = str_output_parser.invoke(response_raw)

In [59]:
print(final_response)

{"response": "I'm sorry to hear that you're experiencing issues with your phone. I completely understand how frustrating that must be. Could you please let me know what specific problems you're encountering? Whether it's performance, battery life, or something else, I'd be happy to help you troubleshoot or find a solution. Your satisfaction is important, and I'm here to assist you in any way I can. Feel free to reach out if there's anything else I can do to resolve this."}
